In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import pyodbc

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pingouin as pg
import scipy as sp
import plotly as py
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.figure_factory as ff
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict,GridSearchCV,StratifiedKFold
import xgboost as xgb
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix,plot_confusion_matrix,accuracy_score,classification_report
#py.offline.init_notebook_mode(connected=True)
import warnings
import tabula
warnings.filterwarnings("ignore")

C:\Users\PSEN\Anaconda3\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning:

The package pingouin is out of date. Your version is 0.4.0, the latest is 0.5.1.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.

C:\Users\PSEN\Anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.



# Model

In [2]:
class Model:
    
    def __init__(self,df):
        self.df=df
    
    # creating X and y variable and building the trian test split
    def train_test(self):
        X=self.df.iloc[:,1]
        y=self.df.iloc[:,0]
        X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.2,random_state=42)
        return [X_train,X_test,y_train,y_test,X,y]


    # stratified K-Fold cross validation
    def cross_val(self):
        val=StratifiedKFold(n_splits=3)
        return val
    
    
    # creating a pipeline to create verctorization, classification model and Stratified k-fold cross validation
    def score(self):
        
        models={'SVC':svm.SVC(),'Random_Forest':RandomForestClassifier(),'XGboost':xgb.XGBClassifier(verbosity=0),'Naive':MultinomialNB(),'Decision_Tree':DecisionTreeClassifier()}
        z={}


        for key,item in models.items():
            text_clf=Pipeline([('tfidf',TfidfVectorizer()),('clf',item)])
            score=cross_val_score(text_clf,self.train_test()[0],self.train_test()[2],cv=self.cross_val(),scoring='accuracy')
            z.update({key:list(score)})
        df_score=pd.DataFrame(z)
        return df_score


    # Creating the confusion matrix
    def matrix(self,model):
        text_clf_svc=Pipeline([('tfidf',TfidfVectorizer()),('clf',model)])
        prediction_svc=cross_val_predict(text_clf_svc,self.train_test()[0],self.train_test()[2],cv=self.cross_val())
        data={'Prediction':list(prediction_svc),'Actual':list(self.train_test()[2].values)}
        df=pd.DataFrame(data)
        confusion_df=pd.crosstab(df.Actual,df.Prediction)

        plt.figure(figsize=(25,10))
        plt.title('Confusion Matrix')
        plt.ylabel('Actal Values')
        plt.xlabel('Predicted Values')
        sns.heatmap(confusion_df, annot=True,fmt='g',cmap='Blues')
        x=plt.show()
        return x
        
        
    # Creating the classification report
    def c_report(self,model):
        text_clf_svc=Pipeline([('tfidf',TfidfVectorizer()),('clf',model)])
        prediction_svc=cross_val_predict(text_clf_svc,self.train_test()[0],self.train_test()[2],cv=self.cross_val())

        report=classification_report(self.train_test()[2], prediction_svc)
        return report     
        



    #Evaluating the model on test set
    def evaluation(self):
        models={'SVC':svm.SVC(),'Random_Forest':RandomForestClassifier(),'XGboost':xgb.XGBClassifier(verbosity=0),'Naive':MultinomialNB(),'Decision_Tree':DecisionTreeClassifier()}
        x=[]
        y=[]

        for key,item in models.items():
    
            text_clf=Pipeline([('tfidf',TfidfVectorizer()),('clf',item)])
            text_clf.fit(self.train_test()[0],self.train_test()[2])
            y_pred=text_clf.predict(self.train_test()[1])
            score=accuracy_score(y_true=self.train_test()[3],y_pred=y_pred)
            x.append(score)
            y.append(key)
        df_score=pd.DataFrame({'model':y,'Accuarcy Score':x})
        return df_score




    #Hyperparameter optimization
    def hyper_param(self):

        model_param={
            'clf__kernel':('linear', 'rbf'), 
            'clf__C':[1,5,10,20], 
            'clf__gamma':['scale','auto'],
            'clf__decision_function_shape':['ovo','ovr']}
                    
        text_clf=Pipeline([('tfidf',TfidfVectorizer()),('clf',svm.SVC())])
        grid=GridSearchCV(text_clf,model_param,cv=self.cross_val(),return_train_score=False)
        grid.fit(self.train_test()[0],self.train_test()[2])
        y_pred=grid.predict(self.train_test()[1])
        accuracy=accuracy_score(self.train_test()[3],y_pred)

        return 'The best score for grid search is {},Accuracy on test set is {}'.format(grid.best_score_,accuracy)             
       

    # Making prediction with the optimized model
    def prediction(self,data):

        model_param={
            'clf__kernel':('linear', 'rbf'), 
            'clf__C':[1,5,10,20], 
            'clf__gamma':['scale','auto'],
            'clf__decision_function_shape':['ovo','ovr']}
                    
        text_clf=Pipeline([('tfidf',TfidfVectorizer()),('clf',svm.SVC())])
        grid=GridSearchCV(text_clf,model_param,cv=self.cross_val(),return_train_score=False)
        grid.fit(self.train_test()[4],self.train_test()[5])
        y_pred=grid.predict(data)  
        return y_pred 



    # data table to validate errors 
    def data(self,model):
        text_clf_svc=Pipeline([('tfidf',TfidfVectorizer()),('clf',model)])
        prediction_svc=cross_val_predict(text_clf_svc,self.train_test()[0],self.train_test()[2],cv=self.cross_val())
        data={'Prediction':list(prediction_svc),'Actual':list(self.train_test()[2].values),'Text':list(self.train_test()[0].values)}
        df=pd.DataFrame(data)
        return df